In [4]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config
from pyeoskit._hello import _eosapi

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)


def set_code(account_name, code):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash == r['code_hash']:
        return

    setcode = {"account":account_name,
               "vmtype": 1,
               "vmversion":0,
               "code":code.hex()
               }
    eosapi.push_action('eosio', 'setcode', setcode, {account_name:'active'})
    
    return True

def set_abi(account, abi):
    db.set_abi(account, abi)
    abi = _eosapi.pack_abi(abi)
    setabi ={'account':account, 'abi':abi.hex()}
    eosapi.push_action('eosio', 'setabi', setabi, {account:'active'})

print('done!')

done!


### idx64_store

```python
def idx64_store(scope: Union[uint64_t, str], table: Union[uint64_t, str], payer: Union[uint64_t, str], id: Union[uint64_t, str], secondary: Union[uint64_t, str]) -> int32_t
```


In [ ]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    primary = 'hello'
    itr = db.idx64_store(scope, table, payer, primary, b'world')
    print(itr)
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

### idx64_find_primary

```python
def idx64_find_primary(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], primary: Union[uint64_t, str]) -> [int32_t, uint64_t]
```

return [iter, secondary]


In [ ]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    primary = 'hello'
    itr, secondary = db.idx64_find_primary(code, scope, table, primary)
    print(itr, n2s(secondary))
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

### idx64_find_secondary

```python
def idx64_find_secondary(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], secondary: Union[uint64_t, str]) -> [int32_t, uint64_t]
```

return [iter, primary]

In [ ]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    secondary = s2n('world')
    itr, primary = db.idx64_find_secondary(code, scope, table, secondary)
    print(itr, n2s(primary))
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

### idx64_update

```python
def idx64_update(iterator: int32_t, payer: Union[uint64_t, str], secondary: Union[uint64_t, str]) -> int32_t
```


In [ ]:
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    primary = 'hello'
    
    itr, secondary = db.idx64_find_primary(code, scope, table, primary)
    print(itr)
    if itr < 0:
        db.idx64_store(scope, table, payer, primary, b'world')
    else:
        data = read_action_data()
        secondary = int.from_bytes(data, 'little')
        db.idx64_update(itr, payer, secondary)
        print(secondary)
'''
abi = ''
name = 'helloworld11'
publish_contract(name, code, abi)
try:
    arg = int.to_bytes(123, 8, 'little')
    r = eosapi.push_action(name, 'sayhello', arg, {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

### idx64_lowerbound

```python
def idx64_lowerbound(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str]) -> [int32_t, uint64_t, uint64_t]
```

return [iter, secondary, primary]

### idx64_upperbound

```python
def idx64_upperbound(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str]) -> [int32_t, uint64_t, uint64_t]
```

return [iter, secondary, primary]


In [5]:
abi = ''
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    data = read_action_data()
    if action == N('destroy'):
        while True:
            itr = db.idx64_end(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.idx64_previous(itr)
            if itr < 0:
                break
            db.idx64_remove(itr)
        print('destroy done!')
    elif action == N('init'):
        db.idx64_store(scope, table, payer, 11, 1)
        db.idx64_store(scope, table, payer, 13, 3)
        db.idx64_store(scope, table, payer, 15, 5)
        db.idx64_store(scope, table, payer, 17, 7)
        print('init done!')
    elif action == N('test'):
        itr, primary, secondary = db.idx64_upperbound(code, scope, table, 3)
        print('upperbound of 3:', primary, secondary)

        itr, primary, secondary = db.idx64_upperbound(code, scope, table, 2)
        print('upperbound of 2:', primary, secondary)

        itr, primary, secondary = db.idx64_lowerbound(code, scope, table, 3)
        print('lowerbound of 3:', primary, secondary)

        itr, primary, secondary = db.idx64_lowerbound(code, scope, table, 2)
        print('lowerbound of 2:', primary, secondary)
'''

name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'destroy', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])

    r = eosapi.push_action(name, 'init', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


destroy done!

init done!



In [6]:
try:
    r = eosapi.push_action(name, 'test', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


upperbound of 3: 15 5
upperbound of 2: 13 3
lowerbound of 3: 13 3
lowerbound of 2: 13 3




### idx64_remove

```python
void idx64_remove(iterator: int32_t) -> int32_t
```

### idx64_next

```python
def idx64_next(iterator: int32_t) -> [int32_t, primary]
```

return [iter, primary]

### idx64_previous

```python
def idx64_previous(iterator: int32_t, uint64_t* primary) -> [int32_t, primary]
```

return [iter, primary]



### idx64_end

```python
def idx64_end(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str]) -> int32_t
```

In [ ]:
abi = ''
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = read_action_data()
    if action == N('destroy'):
        while True:
            itr = db.end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.previous_i64(itr)
            if itr < 0:
                break
            db.remove_i64(itr)

        while True:
            itr = db.idx64_end(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.idx64_previous(itr)
            if itr < 0:
                break
            db.idx64_remove(itr)
        print('destroy done!')
    elif action == N('init'):
        db.idx64_store(scope, table, payer, 11, 1)
        db.idx64_store(scope, table, payer, 13, 3)
        db.idx64_store(scope, table, payer, 15, 5)
        db.idx64_store(scope, table, payer, 17, 7)
        print('init done!')
    elif action == N('test'):
        itr, primary, secondary = db.idx64_upperbound(code, scope, table, 3)
        print('upperbound of 3:', primary, secondary)

        itr, primary, secondary = db.idx64_upperbound(code, scope, table, 2)
        print('upperbound of 2:', primary, secondary)

        itr, primary, secondary = db.idx64_lowerbound(code, scope, table, 3)
        print('lowerbound of 3:', primary, secondary)

        itr, primary, secondary = db.idx64_lowerbound(code, scope, table, 2)
        print('lowerbound of 2:', primary, secondary)
'''

name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'destroy', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])

    r = eosapi.push_action(name, 'init', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


In [ ]:
try:
    r = eosapi.push_action(name, 'test', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

# multi_index implementation

In [114]:
abi = ''
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    data = read_action_data()
    if action == N('destroy'):
        while True:
            itr = db.end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.previous_i64(itr)
            if itr < 0:
                break
            db.remove_i64(itr)

        while True:
            itr = db.idx64_end(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.idx64_previous(itr)
            if itr < 0:
                break
            db.idx64_remove(itr)
        print('destroy done!')
    elif action == N('init'):
        primary, secondary = 11, 1
        db.store_i64(scope, table, payer, primary, b'1')
        db.idx64_store(scope, table, payer, primary, secondary)

        primary, secondary = 13, 3
        db.store_i64(scope, table, payer, primary, b'3')
        db.idx64_store(scope, table, payer, primary, secondary)

        primary, secondary = 15, 5
        db.store_i64(scope, table, payer, primary, b'5')
        db.idx64_store(scope, table, payer, primary, secondary)

        primary, secondary = 17, 7
        db.store_i64(scope, table, payer, primary, b'7')
        db.idx64_store(scope, table, payer, primary, secondary)
        print('init done!')
    elif action == N('test'):
        for secondary in [1, 3, 5, 7]:
            itr, primary = db.idx64_find_secondary(code, scope, table, secondary)
            itr2 = db.find_i64(code, scope, table, primary)
            data = db.get_i64(itr2)
            print('++++++data:', data)
'''

name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'destroy', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])

    r = eosapi.push_action(name, 'init', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


destroy done!

init done!



In [115]:
try:
    r = eosapi.push_action(name, 'test', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


++++++data: b'1'
++++++data: b'3'
++++++data: b'5'
++++++data: b'7'



In [120]:
abi = ''
code = '''
import db
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    data = read_action_data()
    if action == N('destroy'):
        while True:
            itr = db.end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = db.previous_i64(itr)
            if itr < 0:
                break
            db.remove_i64(itr)
        
        table_secondary = s2n(table) & 0xFFFFFFFFFFFFFFF0
        while True:
            itr = db.idx64_end(code, scope, table_secondary+0)
            if itr == -1:
                break
            itr, primary = db.idx64_previous(itr)
            if itr < 0:
                break
            db.idx64_remove(itr)

        while True:
            itr = db.idx64_end(code, scope, table_secondary+1)
            if itr == -1:
                break
            itr, primary = db.idx64_previous(itr)
            if itr < 0:
                break
            db.idx64_remove(itr)

        print('destroy done!')
    elif action == N('init'):
        table_secondary = s2n(table) & 0xFFFFFFFFFFFFFFF0
        primary, secondary, third = 111, 11, 1
        db.store_i64(scope, table, payer, primary, b'1')
        db.idx64_store(scope, table_secondary+0, payer, primary, secondary)
        db.idx64_store(scope, table_secondary+1, payer, primary, third)

        primary, secondary, third = 333, 33, 3
        db.store_i64(scope, table, payer, primary, b'3')
        db.idx64_store(scope, table_secondary+0, payer, primary, secondary)
        db.idx64_store(scope, table_secondary+1, payer, primary, third)

        primary, secondary, third = 555, 55, 5
        db.store_i64(scope, table, payer, primary, b'5')
        db.idx64_store(scope, table_secondary+0, payer, primary, secondary)
        db.idx64_store(scope, table_secondary+1, payer, primary, third)

        primary, secondary, third = 777, 77, 7
        db.store_i64(scope, table, payer, primary, b'7')
        db.idx64_store(scope, table_secondary+0, payer, primary, secondary)
        db.idx64_store(scope, table_secondary+1, payer, primary, third)
        print('init done!')
    elif action == N('test'):
        table_secondary = s2n(table) & 0xFFFFFFFFFFFFFFF0
        for secondary in [11, 33, 55, 77]:
            itr, primary = db.idx64_find_secondary(code, scope, table_secondary, secondary)
            print(itr, primary)
            itr2 = db.find_i64(code, scope, table, primary)
            data = db.get_i64(itr2)
            print('++++++data:', data)

        for secondary in [1, 3, 5, 7]:
            itr, primary = db.idx64_find_secondary(code, scope, table_secondary+1, secondary)
            print(itr, primary)
            itr2 = db.find_i64(code, scope, table, primary)
            data = db.get_i64(itr2)
            print('++++++data:', data)

'''

name = 'helloworld11'
publish_contract(name, code, abi)
try:
    r = eosapi.push_action(name, 'destroy', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])

    r = eosapi.push_action(name, 'init', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


destroy done!

init done!



In [121]:
try:
    r = eosapi.push_action(name, 'test', b'', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)


0 111
++++++data: b'1'
1 333
++++++data: b'3'
2 555
++++++data: b'5'
3 777
++++++data: b'7'
4 111
++++++data: b'1'
5 333
++++++data: b'3'
6 555
++++++data: b'5'
7 777
++++++data: b'7'

